# Import libraries and csv file

In [143]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
import datetime as dt

In [144]:
# Read file in as Pandas dataframe

df = pd.read_csv('kc_house_data.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


# Preproccess and clean datatypes, null values, and value ranges

In [145]:
df['sqft_basement'] = pd.to_numeric(df['sqft_basement'], errors = 'coerce')

df['date'] = pd.to_datetime(df['date'], infer_datetime_format = True)

df['yrs_old'] = [i.year for i in df['date']] - df['yr_built']

df.drop(['date'], axis = 1, inplace = True)

In [146]:
df['view'] = df['view'].fillna(df['view'].mode()[0])

df['waterfront'] = df['waterfront'].fillna(df['waterfront'].mode()[0])

df['renovated'] = [0 if i == 0.0 else 1 for i in df['yr_renovated']]
df.drop(['yr_renovated'], axis = 1, inplace = True)

df['basement'] = [0 if i == 0.0 else 1 for i in df['sqft_basement']]
df.drop(['sqft_basement'], axis = 1, inplace = True)

In [147]:
df.drop(['yr_built', 'id'], axis = 1, inplace = True)

In [148]:
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,zipcode,lat,long,sqft_living15,sqft_lot15,yrs_old,renovated,basement
0,221900.0,3,1.00,1180,5650,1.0,0.0,0.0,3,7,1180,98178,47.5112,-122.257,1340,5650,59,0,0
1,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,98125,47.7210,-122.319,1690,7639,63,1,1
2,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,98028,47.7379,-122.233,2720,8062,82,1,0
3,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,98136,47.5208,-122.393,1360,5000,49,0,1
4,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,98074,47.6168,-122.045,1800,7503,28,0,0


In [149]:
df = df[(df['price'] >= 0) & (df['price'] <= 1000000)]
df = df[(df['bedrooms'] >= 0) & (df['bedrooms'] <= 7)]
df = df[(df['bathrooms'] >= 0) & (df['bathrooms'] <= 5)]

In [150]:
df['dpsf'] = df['price'] / df['sqft_living']

In [151]:
df['dpsf'].head()

0    188.050847
1    209.338521
2    233.766234
3    308.163265
4    303.571429
Name: dpsf, dtype: float64

# Import distance to city feature

In [66]:
from haversine import haversine

seattle = [47.6092, -122.3363]
bellevue = [47.61555, -122.20392]

In [67]:
haversine(seattle, bellevue, unit ='mi')

6.181695095872431

In [68]:
df['lat_long'] = tuple(zip(df.lat, df.long))

In [69]:
seattle_distances = []
for i in df['lat_long']:
    seattle_distances.append(haversine((seattle), (i), unit = 'mi'))
df['distance_from_seattle'] = pd.Series(seattle_distances)

bellevue_distances = []
for i in df['lat_long']:
    bellevue_distances.append(haversine((bellevue), (i), unit = 'mi'))
df['distance_from_bellevue'] = pd.Series(bellevue_distances)

In [70]:
df['distance_from_city'] = df[['distance_from_bellevue', 'distance_from_seattle']].min(axis = 1)

In [71]:
df.drop(['lat', 'long', 'lat_long', 'distance_from_seattle', 'distance_from_bellevue'], axis = 1, inplace = True)

In [72]:
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,zipcode,sqft_living15,sqft_lot15,yrs_old,renovated,basement,distance_from_city
0,221900.0,3,1.00,1180,5650,1.0,0.0,0.0,3,7,1180,98178,1340,5650,59,0,0,7.622786
1,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,98125,1690,7639,63,1,1,7.766476
2,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,98028,2720,8062,82,1,0,8.561145
3,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,98136,1360,5000,49,0,1,6.655340
4,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,98074,1800,7503,28,0,0,7.402273


In [49]:
df.corr()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,zipcode,sqft_living15,sqft_lot15,yrs_old,renovated,basement,distance_from_city
price,1.000000,0.295275,0.440243,0.604644,0.094318,0.269486,0.051813,0.213120,0.033946,0.617743,0.515704,-0.019540,0.549575,0.082325,-0.058760,0.025985,0.153050,-0.009714
bedrooms,0.295275,1.000000,0.499567,0.604720,0.026786,0.157301,-0.027363,0.031535,0.023294,0.330037,0.479849,-0.159746,0.389061,0.024143,-0.173569,-0.013175,0.142414,0.004784
bathrooms,0.440243,0.499567,1.000000,0.709769,0.062676,0.504069,-0.006228,0.080674,-0.139028,0.608200,0.630987,-0.214744,0.524030,0.063067,-0.554380,-0.005848,0.127784,-0.003733
sqft_living,0.604644,0.604720,0.709769,1.000000,0.157876,0.336630,0.000690,0.147725,-0.074536,0.694990,0.849608,-0.211589,0.734272,0.172156,-0.358993,-0.004587,0.168651,-0.005816
sqft_lot,0.094318,0.026786,0.062676,0.157876,1.000000,-0.018346,0.025736,0.081479,-0.002288,0.090518,0.159969,-0.128953,0.140817,0.705859,-0.040458,-0.008173,-0.033794,-0.002941
floors,0.269486,0.157301,0.504069,0.336630,-0.018346,1.000000,-0.006607,-0.018082,-0.278563,0.457574,0.528446,-0.067107,0.267220,-0.022568,-0.515483,-0.010041,-0.278099,-0.017548
waterfront,0.051813,-0.027363,-0.006228,0.000690,0.025736,-0.006607,1.000000,0.282320,0.011941,-0.013079,-0.009791,0.043204,0.005155,0.031845,0.034956,0.018796,0.017433,0.002341
view,0.213120,0.031535,0.080674,0.147725,0.081479,-0.018082,0.282320,1.000000,0.030999,0.114924,0.053130,0.101812,0.168911,0.078011,0.068833,0.024010,0.133400,-0.014288
condition,0.033946,0.023294,-0.139028,-0.074536,-0.002288,-0.278563,0.011941,0.030999,1.000000,-0.174140,-0.179969,-0.007297,-0.122257,0.002989,0.350477,-0.026073,0.123152,0.005276
grade,0.617743,0.330037,0.608200,0.694990,0.090518,0.457574,-0.013079,0.114924,-0.174140,1.000000,0.700805,-0.189264,0.662988,0.100542,-0.499014,-0.018061,0.000098,-0.016431


## Model with Distance to City Feature

In [48]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

outcome = 'price'
predictors = df.drop(['price'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.567
Model:                            OLS   Adj. R-squared:                  0.567
Method:                 Least Squares   F-statistic:                     1446.
Date:                Thu, 18 Feb 2021   Prob (F-statistic):               0.00
Time:                        14:46:52   Log-Likelihood:            -2.4762e+05
No. Observations:               18771   AIC:                         4.953e+05
Df Residuals:                   18753   BIC:                         4.954e+05
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept          -1.191e+07   1.98e+06     -6.031      0.000   -1.58e+07   -8.04e+06
bedrooms           -1.303e+04   1415.908     -9.204      0.000   -1.58e+04   -1.03e+04
bathrooms           2.643e+04   2355.568     11.219      0.000    2.18e+04     3.1e+04
sqft_living           45.1424      4.545      9.932      0.000      36.233      54.051
sqft_lot               0.1522      0.033      4.627      0.000       0.088       0.217
floors              4.635e+04   2623.732     17.665      0.000    4.12e+04    5.15e+04
waterfront          1.035e+05   1.92e+04      5.385      0.000    6.58e+04    1.41e+05
view                1.593e+04   1646.942      9.671      0.000    1.27e+04    1.92e+04
condition           1.976e+04   1563.141     12.640      0.000    1.67e+04    2.28e+04
grade               8.537e+04   1542.276     55.350      0.000    8.23e+04    8.84e+04
sqft_above            10.2762      4.948      2.077      0.038       0.578      19.974
zipcode              114.5795     20.141      5.689      0.000      75.101     154.058
sqft_living15         64.0410      2.611     24.524      0.000      58.922      69.159
sqft_lot15            -0.2342      0.051     -4.637      0.000      -0.333      -0.135
yrs_old             2572.7238     47.977     53.625      0.000    2478.685    2666.762
renovated           1728.6000   2353.455      0.734      0.463   -2884.384    6341.584
basement            3.379e+04   3544.040      9.535      0.000    2.68e+04    4.07e+04
distance_from_city   -10.4750    151.897     -0.069      0.945    -308.207     287.257
==============================================================================
Omnibus:                      476.549   Durbin-Watson:                   1.968
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              573.902
Skew:                           0.337   Prob(JB):                    2.39e-125
Kurtosis:                       3.530   Cond. No.                     2.10e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.1e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [50]:
# Drop column due to high p-value

df.drop(['distance_from_city'], axis = 1, inplace = True)

In [51]:
outcome = 'price'
predictors = df.drop(['price'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.567
Model:                            OLS   Adj. R-squared:                  0.567
Method:                 Least Squares   F-statistic:                     1646.
Date:                Thu, 18 Feb 2021   Prob (F-statistic):               0.00
Time:                        14:48:42   Log-Likelihood:            -2.6530e+05
No. Observations:               20116   AIC:                         5.306e+05
Df Residuals:                   20099   BIC:                         5.308e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -1.439e+07   1.92e+06     -7.499      0.000   -1.81e+07   -1.06e+07
bedrooms      -1.361e+04   1376.105     -9.892      0.000   -1.63e+04   -1.09e+04
bathrooms      2.536e+04   2264.360     11.200      0.000    2.09e+04    2.98e+04
sqft_living      46.1319      4.417     10.445      0.000      37.475      54.789
sqft_lot          0.1563      0.032      4.832      0.000       0.093       0.220
floors         4.979e+04   2436.079     20.441      0.000     4.5e+04    5.46e+04
waterfront      1.03e+05   1.91e+04      5.386      0.000    6.55e+04    1.41e+05
view           1.565e+04   1612.038      9.708      0.000    1.25e+04    1.88e+04
condition      1.989e+04   1554.384     12.794      0.000    1.68e+04    2.29e+04
grade          8.647e+04   1470.198     58.813      0.000    8.36e+04    8.93e+04
sqft_above       10.7462      4.761      2.257      0.024       1.415      20.078
zipcode         139.8211     19.564      7.147      0.000     101.475     178.168
sqft_living15    59.7966      2.491     24.001      0.000      54.913      64.680
sqft_lot15       -0.2213      0.049     -4.486      0.000      -0.318      -0.125
yrs_old        2470.2621     45.902     53.816      0.000    2380.291    2560.234
renovated      2811.9226   2268.219      1.240      0.215   -1633.972    7257.817
basement       3.563e+04   3395.207     10.495      0.000     2.9e+04    4.23e+04
==============================================================================
Omnibus:                      521.072   Durbin-Watson:                   1.964
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              629.514
Skew:                           0.340   Prob(JB):                    2.01e-137
Kurtosis:                       3.537   Cond. No.                     2.11e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.11e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [52]:
# drop column due to correlation with sqft features

df.drop(['grade'], axis = 1, inplace = True)

In [53]:
outcome = 'price'
predictors = df.drop(['price'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.492
Method:                 Least Squares   F-statistic:                     1301.
Date:                Thu, 18 Feb 2021   Prob (F-statistic):               0.00
Time:                        14:50:02   Log-Likelihood:            -2.6689e+05
No. Observations:               20116   AIC:                         5.338e+05
Df Residuals:                   20100   BIC:                         5.339e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -2.072e+07   2.07e+06     -9.991      0.000   -2.48e+07   -1.67e+07
bedrooms       -2.36e+04   1478.403    -15.961      0.000   -2.65e+04   -2.07e+04
bathrooms      3.421e+04   2446.002     13.985      0.000    2.94e+04     3.9e+04
sqft_living      63.8724      4.770     13.390      0.000      54.522      73.222
sqft_lot          0.1575      0.035      4.499      0.000       0.089       0.226
floors         6.707e+04   2618.076     25.618      0.000    6.19e+04    7.22e+04
waterfront      6.92e+04   2.07e+04      3.343      0.001    2.86e+04     1.1e+05
view           2.013e+04   1743.257     11.547      0.000    1.67e+04    2.35e+04
condition      2.142e+04   1682.551     12.731      0.000    1.81e+04    2.47e+04
sqft_above       47.6611      5.109      9.329      0.000      37.647      57.675
zipcode         209.2636     21.141      9.898      0.000     167.825     250.702
sqft_living15    97.7800      2.605     37.536      0.000      92.674     102.886
sqft_lot15       -0.3076      0.053     -5.764      0.000      -0.412      -0.203
yrs_old        1951.7820     48.769     40.021      0.000    1856.191    2047.373
renovated      4036.5300   2455.488      1.644      0.100    -776.428    8849.488
basement        5.53e+04   3657.805     15.118      0.000    4.81e+04    6.25e+04
==============================================================================
Omnibus:                      495.462   Durbin-Watson:                   1.964
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              538.181
Skew:                           0.378   Prob(JB):                    1.37e-117
Kurtosis:                       3.264   Cond. No.                     2.11e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.11e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [54]:
# drop column due to high p-value

df.drop(['renovated'], axis = 1, inplace = True)

In [55]:
outcome = 'price'
predictors = df.drop(['price'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.492
Method:                 Least Squares   F-statistic:                     1394.
Date:                Thu, 18 Feb 2021   Prob (F-statistic):               0.00
Time:                        14:50:39   Log-Likelihood:            -2.6689e+05
No. Observations:               20116   AIC:                         5.338e+05
Df Residuals:                   20101   BIC:                         5.339e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -2.069e+07   2.07e+06     -9.974      0.000   -2.48e+07   -1.66e+07
bedrooms      -2.365e+04   1478.083    -16.001      0.000   -2.65e+04   -2.08e+04
bathrooms      3.439e+04   2443.622     14.072      0.000    2.96e+04    3.92e+04
sqft_living      63.9091      4.770     13.397      0.000      54.559      73.259
sqft_lot          0.1569      0.035      4.482      0.000       0.088       0.226
floors         6.711e+04   2618.085     25.633      0.000     6.2e+04    7.22e+04
waterfront     6.957e+04   2.07e+04      3.361      0.001     2.9e+04     1.1e+05
view           2.016e+04   1743.219     11.566      0.000    1.67e+04    2.36e+04
condition      2.125e+04   1679.338     12.653      0.000     1.8e+04    2.45e+04
sqft_above       47.7082      5.109      9.338      0.000      37.694      57.723
zipcode         208.9113     21.141      9.882      0.000     167.473     250.350
sqft_living15    97.6744      2.604     37.505      0.000      92.570     102.779
sqft_lot15       -0.3072      0.053     -5.755      0.000      -0.412      -0.203
yrs_old        1960.9875     48.448     40.476      0.000    1866.025    2055.950
basement       5.528e+04   3657.933     15.111      0.000    4.81e+04    6.24e+04
==============================================================================
Omnibus:                      496.647   Durbin-Watson:                   1.963
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              539.646
Skew:                           0.379   Prob(JB):                    6.57e-118
Kurtosis:                       3.265   Cond. No.                     2.11e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.11e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [56]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [58]:
dfi = df.drop(['price'], axis = 1)

for i in range(len(dfi.columns[:-1])):
    v = vif(np.matrix(dfi[:-1]), i)
    print('Variance inflation factor for {}: {}'.format(dfi.columns[i], round(v, 2)))

Variance inflation factor for bedrooms: 26.25
Variance inflation factor for bathrooms: 28.41
Variance inflation factor for sqft_living: 102.34
Variance inflation factor for sqft_lot: 2.27
Variance inflation factor for floors: 16.77
Variance inflation factor for waterfront: 1.09
Variance inflation factor for view: 1.26
Variance inflation factor for condition: 34.16
Variance inflation factor for sqft_above: 90.16
Variance inflation factor for zipcode: 76.03
Variance inflation factor for sqft_living15: 27.31
Variance inflation factor for sqft_lot15: 2.48
Variance inflation factor for yrs_old: 6.1


### Even after columns with high p-values are dropped, there remains several columns with high vif due to the correlation between square footage and number of rooms

## Model with Zip to School District Conversion Feature

### Must skip executing model with distance to city cells to run successfully

In [152]:
def group_zips(z):
    
    if z in [98198, 98023, 98003, 98001, 98002, 98092]:
        return 'FederalWay_Auburn'
    
    elif z in [98032, 98031, 98030, 98042, 98058, 98178, 98056, 98055, 98146, 98166, 98148, 98188, 98038, 98022, 98010]:
        return 'Kent_Renton_Highline_Tahoma_Enumclaw'
    
    
    elif z in [98177, 98133, 98125, 98155, 98019, 98014]:
        return 'Shoreline_Riverview'
    
    elif z in [98024, 98065, 98045, 98070]:
        return 'Snoqualmie_Vashon'

    elif z in [98117, 98103, 98115, 98107, 98105, 98199, 98119, 98109, 98102, 
               98112, 98122, 98116, 98136, 98126,
               98106, 98144, 98108, 98118, 98028, 98011, 98072, 98077]:
        return 'Seattle_Northshore'
    
    elif z in [98034, 98052, 98053, 98033, 98074, 98059, 98027, 98075, 98029]:
        return 'LakeWashington_Issaquah'
    
    elif z in [98040]:
        return 'MercerIsland'
    
    elif z in [98039, 98004, 98005, 98007, 98008, 98006]:
        return 'Bellevue'
    
    elif z in [98168]:
        return 'Tukwila'

In [153]:
df['zipcode'] = df['zipcode'].map(group_zips)

In [154]:
df['zipcode'].head()

0    Kent_Renton_Highline_Tahoma_Enumclaw
1                     Shoreline_Riverview
2                      Seattle_Northshore
3                      Seattle_Northshore
4                 LakeWashington_Issaquah
Name: zipcode, dtype: object

In [158]:
df.sort_values(['price']).groupby('zipcode').mean()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,lat,long,sqft_living15,sqft_lot15,yrs_old,renovated,basement,dpsf
zipcode,,,,,,,,,,,,,,,,,,,
Bellevue,660437.058989,3.697566,2.161049,2223.596442,11614.596442,1.292603,0.000936,0.178839,3.691948,7.941011,1812.640449,47.592891,-122.149780,2246.306180,10888.213483,44.984082,0.193820,0.467228,312.528401
FederalWay_Auburn,290395.965394,3.376590,2.017303,1925.243257,14902.447328,1.376845,0.005089,0.181170,3.407125,7.408651,1710.157761,47.320145,-122.282209,1870.201018,13364.426972,36.364377,0.184224,0.313995,155.903659
Kent_Renton_Highline_Tahoma_Enumclaw,336981.576440,3.377578,2.024176,1927.878644,18922.798530,1.416212,0.004266,0.160465,3.438730,7.295805,1721.469543,47.414252,-122.169844,1871.308130,14792.037213,38.159279,0.202181,0.291064,182.352847
LakeWashington_Issaquah,581100.134263,3.519278,2.331214,2304.694011,15453.378452,1.600492,0.000273,0.085316,3.300246,8.044299,2093.466776,47.622079,-122.095738,2281.501504,13837.112387,27.529943,0.191687,0.298879,262.179017
MercerIsland,815539.904762,3.755102,2.234694,2388.612245,12203.034014,1.285714,0.000000,0.210884,3.836735,8.210884,1956.530612,47.561840,-122.225551,2588.775510,11456.965986,49.666667,0.204082,0.469388,359.968101
Seattle_Northshore,525258.529572,3.105111,1.909916,1753.696721,7460.962713,1.523947,0.000321,0.185471,3.415301,7.380906,1427.900836,47.637215,-122.314937,1696.753616,6821.848762,57.945677,0.224687,0.538251,320.096694
Shoreline_Riverview,429792.595824,3.224839,1.862286,1809.261777,16894.370450,1.364829,0.000000,0.148287,3.400964,7.252141,1527.662741,47.733198,-122.256843,1745.940578,15050.536938,48.674518,0.230728,0.400964,252.051580
Snoqualmie_Vashon,473009.198288,3.281027,2.230742,2199.485021,44187.266762,1.631241,0.025678,0.353780,3.209700,7.570613,2074.375178,47.497600,-121.933402,2118.325250,33092.248217,28.636234,0.195435,0.186876,226.695011
Tukwila,240328.371747,2.996283,1.433086,1468.624535,11269.843866,1.159851,0.000000,0.014870,3.245353,6.509294,1234.089219,47.489700,-122.307877,1471.680297,9744.022305,61.579926,0.193309,0.386617,175.372532


In [101]:
df.groupby('zipcode', as_index = False)['price'].mean()

,zipcode,price
0,Bellevue,660437.058989
1,FederalWay_Auburn,290395.965394
2,Kent_Renton_Highline_Tahoma_Enumclaw,336981.576440
3,LakeWashington_Issaquah,581100.134263
4,MercerIsland,815539.904762
5,Seattle_Northshore,525258.529572
6,Shoreline_Riverview,429792.595824
7,Snoqualmie_Vashon,473009.198288
8,Tukwila,240328.371747


In [12]:
df.groupby('zipcode', as_index = False)['sqft_living'].mean()

,zipcode,sqft_living
0,Bellevue,2223.596442
1,FederalWay/Auburn,1925.243257
2,Kent/Renton/Highline/Tahoma/Enumclaw,1927.878644
3,LakeWashington/Issaquah,2304.694011
4,MercerIsland,2388.612245
5,Seattle/Northshore,1753.696721
6,Shoreline/Riverview,1809.261777
7,Snoqualmie/Vashon,2199.485021
8,Tukwila,1468.624535


In [16]:
df.groupby('zipcode', as_index = False)['dpsf'].mean()

,zipcode,dpsf
0,Bellevue,312.528401
1,FederalWay/Auburn,155.903659
2,Kent/Renton/Highline/Tahoma/Enumclaw,182.352847
3,LakeWashington/Issaquah,262.179017
4,MercerIsland,359.968101
5,Seattle/Northshore,320.096694
6,Shoreline/Riverview,252.051580
7,Snoqualmie/Vashon,226.695011
8,Tukwila,175.372532


In [17]:
df.groupby('zipcode', as_index = False)['bedrooms'].mean()

,zipcode,bedrooms
0,Bellevue,3.697566
1,FederalWay/Auburn,3.376590
2,Kent/Renton/Highline/Tahoma/Enumclaw,3.377578
3,LakeWashington/Issaquah,3.519278
4,MercerIsland,3.755102
5,Seattle/Northshore,3.105111
6,Shoreline/Riverview,3.224839
7,Snoqualmie/Vashon,3.281027
8,Tukwila,2.996283


In [18]:
df.groupby('zipcode', as_index = False)['bathrooms'].mean()

,zipcode,bathrooms
0,Bellevue,2.161049
1,FederalWay/Auburn,2.017303
2,Kent/Renton/Highline/Tahoma/Enumclaw,2.024176
3,LakeWashington/Issaquah,2.331214
4,MercerIsland,2.234694
5,Seattle/Northshore,1.909916
6,Shoreline/Riverview,1.862286
7,Snoqualmie/Vashon,2.230742
8,Tukwila,1.433086


In [19]:
df.groupby('zipcode', as_index = False)['yrs_old'].mean()

,zipcode,yrs_old
0,Bellevue,44.984082
1,FederalWay/Auburn,36.364377
2,Kent/Renton/Highline/Tahoma/Enumclaw,38.159279
3,LakeWashington/Issaquah,27.529943
4,MercerIsland,49.666667
5,Seattle/Northshore,57.945677
6,Shoreline/Riverview,48.674518
7,Snoqualmie/Vashon,28.636234
8,Tukwila,61.579926


In [20]:
df.groupby('zipcode', as_index = False)['grade'].mean()

,zipcode,grade
0,Bellevue,7.941011
1,FederalWay/Auburn,7.408651
2,Kent/Renton/Highline/Tahoma/Enumclaw,7.295805
3,LakeWashington/Issaquah,8.044299
4,MercerIsland,8.210884
5,Seattle/Northshore,7.380906
6,Shoreline/Riverview,7.252141
7,Snoqualmie/Vashon,7.570613
8,Tukwila,6.509294


In [124]:
from sklearn.preprocessing import OneHotEncoder
ohc = OneHotEncoder()
ohe = ohc.fit_transform(df.zipcode.values.reshape(-1,1)).toarray()
dfOneHot = pd.DataFrame(ohe, columns = ['SD_' + str(ohc.categories_[0][i]) for i in range(len(ohc.categories_[0]))])
dfh = pd.concat([df, dfOneHot], axis = 1)
dfh.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,dpsf,SD_Bellevue,SD_FederalWay_Auburn,SD_Kent_Renton_Highline_Tahoma_Enumclaw,SD_LakeWashington_Issaquah,SD_MercerIsland,SD_Seattle_Northshore,SD_Shoreline_Riverview,SD_Snoqualmie_Vashon,SD_Tukwila
0,221900.0,3.0,1.00,1180.0,5650.0,1.0,0.0,0.0,3.0,7.0,...,188.050847,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,538000.0,3.0,2.25,2570.0,7242.0,2.0,0.0,0.0,3.0,7.0,...,209.338521,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,180000.0,2.0,1.00,770.0,10000.0,1.0,0.0,0.0,3.0,6.0,...,233.766234,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,604000.0,4.0,3.00,1960.0,5000.0,1.0,0.0,0.0,5.0,7.0,...,308.163265,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,510000.0,3.0,2.00,1680.0,8080.0,1.0,0.0,0.0,3.0,8.0,...,303.571429,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [125]:
dfh.drop(['zipcode'], axis = 1, inplace = True)

In [126]:
dfh.isnull().sum()

price                                      1345
bedrooms                                   1345
bathrooms                                  1345
sqft_living                                1345
sqft_lot                                   1345
floors                                     1345
waterfront                                 1345
view                                       1345
condition                                  1345
grade                                      1345
sqft_above                                 1345
lat                                        1345
long                                       1345
sqft_living15                              1345
sqft_lot15                                 1345
yrs_old                                    1345
renovated                                  1345
basement                                   1345
dpsf                                       1345
SD_Bellevue                                1345
SD_FederalWay_Auburn                    

In [127]:
# remove null values

dfh.dropna(inplace = True)

In [128]:
dfh.isnull().sum()

price                                      0
bedrooms                                   0
bathrooms                                  0
sqft_living                                0
sqft_lot                                   0
floors                                     0
waterfront                                 0
view                                       0
condition                                  0
grade                                      0
sqft_above                                 0
lat                                        0
long                                       0
sqft_living15                              0
sqft_lot15                                 0
yrs_old                                    0
renovated                                  0
basement                                   0
dpsf                                       0
SD_Bellevue                                0
SD_FederalWay_Auburn                       0
SD_Kent_Renton_Highline_Tahoma_Enumclaw    0
SD_LakeWas

In [129]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [130]:
outcome = 'price'
predictors = dfh.drop(['price'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=dfh).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.908
Model:                            OLS   Adj. R-squared:                  0.908
Method:                 Least Squares   F-statistic:                     7113.
Date:                Sun, 21 Feb 2021   Prob (F-statistic):               0.00
Time:                        02:01:14   Log-Likelihood:            -2.3309e+05
No. Observations:               18771   AIC:                         4.662e+05
Df Residuals:                   18744   BIC:                         4.664e+05
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
===========================================================================================================
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
Intercept                               -1.173e+06    4.5e+05     -2.607      0.009   -2.05e+06   -2.91e+05
bedrooms                                 8036.3325    657.583     12.221      0.000    6747.409    9325.256
bathrooms                                1.086e+04   1088.108      9.979      0.000    8725.377     1.3e+04
sqft_living                               156.5164      2.168     72.182      0.000     152.266     160.767
sqft_lot                                    0.0318      0.015      2.086      0.037       0.002       0.062
floors                                   6657.2927   1218.303      5.464      0.000    4269.308    9045.277
waterfront                               2.245e+04   8893.091      2.525      0.012    5021.789    3.99e+04
view                                     9772.0491    764.841     12.777      0.000    8272.891    1.13e+04
condition                                1.244e+04    720.641     17.261      0.000     1.1e+04    1.39e+04
grade                                    2.675e+04    751.552     35.598      0.000    2.53e+04    2.82e+04
sqft_above                                 24.7058      2.291     10.782      0.000      20.214      29.197
lat                                      1.238e+05   3745.802     33.038      0.000    1.16e+05    1.31e+05
long                                     4.207e+04   3810.643     11.041      0.000    3.46e+04    4.95e+04
sqft_living15                               7.6865      1.230      6.249      0.000       5.275      10.098
sqft_lot15                                 -0.0530      0.023     -2.265      0.024      -0.099      -0.007
yrs_old                                   405.2937     23.990     16.894      0.000     358.271     452.316
renovated                                2725.9641   1085.831      2.510      0.012     597.637    4854.291
basement                                 2.694e+04   1637.326     16.452      0.000    2.37e+04    3.01e+04
dpsf                                     1324.3548      6.286    210.699      0.000    1312.035    1336.675
SD_Bellevue                             -1.286e+05      5e+04     -2.569      0.010   -2.27e+05   -3.05e+04
SD_FederalWay_Auburn                    -1.321e+05      5e+04     -2.641      0.008    -2.3e+05   -3.41e+04
SD_Kent_Renton_Highline_Tahoma_Enumclaw -1.324e+05      5e+04     -2.649      0.008    -2.3e+05   -3.44e+04
SD_LakeWashington_Issaquah              -1.312e+05      5e+04     -2.624      0.009   -2.29e+05   -3.32e+04
SD_MercerIsland                         -1.277e+05   5.02e+04     -2.544      0.011   -2.26e+05   -2.93e+04
SD_Seattle_Northshore                   -1.311e+05      5e+04     -2.623      0.009   -2.29e+05   -3.31e+04
SD_Shoreline_Riverview                  -1.313e+05      5e+

In [131]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [138]:
dfhi = dfh.drop(['price'], axis = 1)

for i in range(len(dfhi.columns[:-1])):
    v = vif(np.matrix(dfhi[:-1]), i)
    print('Variance inflation factor for {}: {}'.format(dfhi.columns[i], round(v, 2)))

Variance inflation factor for sqft_living: 1.57
Variance inflation factor for floors: 1.58
Variance inflation factor for waterfront: 1.09
Variance inflation factor for view: 1.16
Variance inflation factor for yrs_old: 1.54
Variance inflation factor for renovated: 1.01
Variance inflation factor for basement: 1.26
Variance inflation factor for dpsf: 1.28
Variance inflation factor for SD_Bellevue: 2.69
Variance inflation factor for SD_FederalWay_Auburn: 4.19
Variance inflation factor for SD_Kent_Renton_Highline_Tahoma_Enumclaw: 7.76
Variance inflation factor for SD_LakeWashington_Issaquah: 6.93
Variance inflation factor for SD_MercerIsland: 1.25
Variance inflation factor for SD_Seattle_Northshore: 11.11
Variance inflation factor for SD_Shoreline_Riverview: 3.99
Variance inflation factor for SD_Snoqualmie_Vashon: 2.13


In [133]:
# drop column due to p-value

dfh.drop(['lat', 'long'], axis = 1, inplace = True)

In [134]:
dfh.drop(['grade', 'sqft_above'], axis = 1, inplace = True)

In [135]:
dfh.drop(['sqft_lot', 'sqft_living15', 'sqft_lot15'], axis = 1, inplace = True)

In [80]:
dfh.drop(['dpsf', 'floors', 'renovated', 'basement'], axis = 1, inplace = True)

In [136]:
dfh.drop(['bathrooms', 'bedrooms'], axis = 1, inplace = True)

In [85]:
dfh.drop(['SD_Bellevue'], axis = 1, inplace = True)

In [137]:
dfh.drop(['condition'], axis = 1, inplace = True)

In [140]:
dfh.drop(['yrs_old', 'waterfront'], axis = 1, inplace = True)

In [141]:
dfhi = dfh.drop(['price'], axis = 1)

for i in range(len(dfhi.columns[:-1])):
    v = vif(np.matrix(dfhi[:-1]), i)
    print('Variance inflation factor for {}: {}'.format(dfhi.columns[i], round(v, 2)))

Variance inflation factor for sqft_living: 1.53
Variance inflation factor for floors: 1.38
Variance inflation factor for view: 1.06
Variance inflation factor for renovated: 1.0
Variance inflation factor for basement: 1.24
Variance inflation factor for dpsf: 1.17
Variance inflation factor for SD_Bellevue: 2.28
Variance inflation factor for SD_FederalWay_Auburn: 3.42
Variance inflation factor for SD_Kent_Renton_Highline_Tahoma_Enumclaw: 6.09
Variance inflation factor for SD_LakeWashington_Issaquah: 5.51
Variance inflation factor for SD_MercerIsland: 1.19
Variance inflation factor for SD_Seattle_Northshore: 8.66
Variance inflation factor for SD_Shoreline_Riverview: 3.27
Variance inflation factor for SD_Snoqualmie_Vashon: 1.87


In [142]:
outcome = 'price'
predictors = dfh.drop(['price'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=dfh).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.892
Model:                            OLS   Adj. R-squared:                  0.891
Method:                 Least Squares   F-statistic:                 1.101e+04
Date:                Sun, 21 Feb 2021   Prob (F-statistic):               0.00
Time:                        02:04:27   Log-Likelihood:            -2.3463e+05
No. Observations:               18771   AIC:                         4.693e+05
Df Residuals:                   18756   BIC:                         4.694e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
===========================================================================================================
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
Intercept                               -3.376e+05   2271.770   -148.596      0.000   -3.42e+05   -3.33e+05
sqft_living                               223.4907      0.787    284.032      0.000     221.948     225.033
floors                                   1.371e+04   1088.461     12.596      0.000    1.16e+04    1.58e+04
view                                     6466.9605    774.439      8.351      0.000    4948.991    7984.930
renovated                                2505.1930   1168.257      2.144      0.032     215.304    4795.082
basement                                 1.417e+04   1080.551     13.116      0.000    1.21e+04    1.63e+04
dpsf                                     1505.7780      5.112    294.563      0.000    1495.758    1515.798
SD_Bellevue                             -3.672e+04   2016.489    -18.208      0.000   -4.07e+04   -3.28e+04
SD_FederalWay_Auburn                    -4.019e+04   1595.702    -25.188      0.000   -4.33e+04   -3.71e+04
SD_Kent_Renton_Highline_Tahoma_Enumclaw -3.932e+04   1247.644    -31.513      0.000   -4.18e+04   -3.69e+04
SD_LakeWashington_Issaquah              -3.805e+04   1303.960    -29.180      0.000   -4.06e+04   -3.55e+04
SD_MercerIsland                         -3.307e+04   4982.848     -6.636      0.000   -4.28e+04   -2.33e+04
SD_Seattle_Northshore                   -3.841e+04   1136.830    -33.790      0.000   -4.06e+04   -3.62e+04
SD_Shoreline_Riverview                  -3.879e+04   1622.242    -23.912      0.000    -4.2e+04   -3.56e+04
SD_Snoqualmie_Vashon                    -3.875e+04   2440.865    -15.876      0.000   -4.35e+04    -3.4e+04
SD_Tukwila                              -3.428e+04   3762.882     -9.109      0.000   -4.17e+04   -2.69e+04
==============================================================================
Omnibus:                     5913.227   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            32095.705
Skew:                          -1.415   Prob(JB):                         0.00
Kurtosis:                       8.747   Cond. No.                     1.58e+19
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.26e-28. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [92]:
# drop columns with high p-value

dfh.drop(['SD_Tukwila', 'SD_VashonIsland', 'SD_SnoqualmieValley', 'SD_Riverview', 'SD_MercerIsland', 'SD_Highline', 'SD_Auburn'], axis = 1, inplace = True)

In [93]:
dfhi = dfh.drop(['price'], axis = 1)

for i in range(len(dfhi.columns[:-1])):
    v = vif(np.matrix(dfhi[:-1]), i)
    print('Variance inflation factor for {}: {}'.format(dfhi.columns[i], round(v, 2)))

Variance inflation factor for bedrooms: 23.83
Variance inflation factor for bathrooms: 26.87
Variance inflation factor for sqft_living: 28.92
Variance inflation factor for sqft_lot: 2.26
Variance inflation factor for floors: 12.73
Variance inflation factor for waterfront: 1.1
Variance inflation factor for view: 1.25
Variance inflation factor for condition: 21.77
Variance inflation factor for sqft_living15: 22.79
Variance inflation factor for sqft_lot15: 2.49
Variance inflation factor for yrs_old: 5.78
Variance inflation factor for basement: 2.25
Variance inflation factor for SD_Bellevue: 1.36
Variance inflation factor for SD_Enumclaw: 1.11
Variance inflation factor for SD_FederalWay: 1.48
Variance inflation factor for SD_Issaquah: 1.49
Variance inflation factor for SD_Kent: 1.56
Variance inflation factor for SD_LakeWashington: 1.75
Variance inflation factor for SD_Northshore: 1.31
Variance inflation factor for SD_Renton: 1.32
Variance inflation factor for SD_Seattle: 2.8
Variance infla

In [94]:
outcome = 'price'
predictors = dfh.drop(['price'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=dfh).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.492
Method:                 Least Squares   F-statistic:                     792.0
Date:                Thu, 18 Feb 2021   Prob (F-statistic):               0.00
Time:                        15:30:13   Log-Likelihood:            -2.4911e+05
No. Observations:               18771   AIC:                         4.983e+05
Df Residuals:                   18747   BIC:                         4.985e+05
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept         -1.917e+05   9143.097    -20.964      0.000    -2.1e+05   -1.74e+05
bedrooms          -2.337e+04   1518.826    -15.390      0.000   -2.64e+04   -2.04e+04
bathrooms          3.442e+04   2539.379     13.555      0.000    2.94e+04    3.94e+04
sqft_living          99.5593      2.750     36.200      0.000      94.168     104.950
sqft_lot              0.1389      0.036      3.904      0.000       0.069       0.209
floors             7.189e+04   2646.838     27.162      0.000    6.67e+04    7.71e+04
waterfront          7.45e+04   2.08e+04      3.582      0.000    3.37e+04    1.15e+05
view               2.028e+04   1762.447     11.505      0.000    1.68e+04    2.37e+04
condition          1.777e+04   1668.520     10.652      0.000    1.45e+04     2.1e+04
sqft_living15       106.0445      2.645     40.090      0.000     100.860     111.229
sqft_lot15           -0.3377      0.055     -6.182      0.000      -0.445      -0.231
yrs_old            2191.7981     48.406     45.280      0.000    2096.918    2286.678
basement           2.842e+04   2457.375     11.566      0.000    2.36e+04    3.32e+04
SD_Bellevue       -9585.2011   5254.736     -1.824      0.068   -1.99e+04     714.557
SD_Enumclaw       -2.034e+04   8495.655     -2.394      0.017    -3.7e+04   -3682.895
SD_FederalWay     -1.086e+04   4775.665     -2.275      0.023   -2.02e+04   -1503.971
SD_Issaquah       -3888.4761   4734.442     -0.821      0.411   -1.32e+04    5391.460
SD_Kent           -1.048e+04   4547.838     -2.305      0.021   -1.94e+04   -1567.688
SD_LakeWashington -6303.0194   4184.183     -1.506      0.132   -1.45e+04    1898.357
SD_Northshore     -7202.6760   5582.613     -1.290      0.197   -1.81e+04    3739.751
SD_Renton         -1.087e+04   5532.470     -1.965      0.049   -2.17e+04     -25.079
SD_Seattle        -4935.6334   3447.228     -1.432      0.152   -1.17e+04    1821.246
SD_Shoreline      -7333.2813   4628.416     -1.584      0.113   -1.64e+04    1738.834
SD_Tahoma         -5274.4597   6597.231     -0.799      0.424   -1.82e+04    7656.710
==============================================================================
Omnibus:                      392.696   Durbin-Watson:                   1.965
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              422.493
Skew:                           0.346   Prob(JB):                     1.81e-92
Kurtosis:                       3.250   Cond. No.                     1.01e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.01e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [95]:
dfh.drop(['bedrooms', 'bathrooms', 'sqft_living15'], axis = 1, inplace = True)

In [96]:
outcome = 'price'
predictors = dfh.drop(['price'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=dfh).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.437
Model:                            OLS   Adj. R-squared:                  0.437
Method:                 Least Squares   F-statistic:                     729.0
Date:                Thu, 18 Feb 2021   Prob (F-statistic):               0.00
Time:                        15:45:35   Log-Likelihood:            -2.5008e+05
No. Observations:               18771   AIC:                         5.002e+05
Df Residuals:                   18750   BIC:                         5.004e+05
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept         -8.167e+04   8444.502     -9.671      0.000   -9.82e+04   -6.51e+04
sqft_living         157.8647      1.781     88.640      0.000     154.374     161.356
sqft_lot              0.1135      0.037      3.031      0.002       0.040       0.187
floors             7.595e+04   2640.193     28.766      0.000    7.08e+04    8.11e+04
waterfront         5.622e+04   2.19e+04      2.568      0.010    1.33e+04    9.91e+04
view               3.267e+04   1833.116     17.824      0.000    2.91e+04    3.63e+04
condition          1.453e+04   1747.111      8.315      0.000    1.11e+04     1.8e+04
sqft_lot15           -0.1877      0.057     -3.274      0.001      -0.300      -0.075
yrs_old            1662.6288     46.697     35.605      0.000    1571.099    1754.159
basement            1.75e+04   2492.445      7.023      0.000    1.26e+04    2.24e+04
SD_Bellevue       -9019.7541   5533.702     -1.630      0.103   -1.99e+04    1826.804
SD_Enumclaw       -2.236e+04   8945.728     -2.499      0.012   -3.99e+04   -4821.631
SD_FederalWay     -8612.1206   5028.843     -1.713      0.087   -1.85e+04    1244.867
SD_Issaquah       -5043.4876   4985.700     -1.012      0.312   -1.48e+04    4728.936
SD_Kent           -1.064e+04   4789.269     -2.222      0.026      -2e+04   -1253.401
SD_LakeWashington -4787.6680   4406.131     -1.087      0.277   -1.34e+04    3848.747
SD_Northshore     -6109.3090   5878.924     -1.039      0.299   -1.76e+04    5413.913
SD_Renton         -1.174e+04   5826.139     -2.014      0.044   -2.32e+04    -315.428
SD_Seattle        -5485.4916   3630.058     -1.511      0.131   -1.26e+04    1629.750
SD_Shoreline      -5310.2801   4873.844     -1.090      0.276   -1.49e+04    4242.895
SD_Tahoma         -3628.5865   6947.299     -0.522      0.601   -1.72e+04    9988.748
==============================================================================
Omnibus:                      360.606   Durbin-Watson:                   1.962
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              381.324
Skew:                           0.344   Prob(JB):                     1.57e-83
Kurtosis:                       3.114   Cond. No.                     1.01e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.01e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [97]:
dfhi = dfh.drop(['price'], axis = 1)

for i in range(len(dfhi.columns[:-1])):
    v = vif(np.matrix(dfhi[:-1]), i)
    print('Variance inflation factor for {}: {}'.format(dfhi.columns[i], round(v, 2)))

Variance inflation factor for sqft_living: 11.22
Variance inflation factor for sqft_lot: 2.26
Variance inflation factor for floors: 10.16
Variance inflation factor for waterfront: 1.1
Variance inflation factor for view: 1.22
Variance inflation factor for condition: 17.6
Variance inflation factor for sqft_lot15: 2.47
Variance inflation factor for yrs_old: 5.01
Variance inflation factor for basement: 2.09
Variance inflation factor for SD_Bellevue: 1.35
Variance inflation factor for SD_Enumclaw: 1.11
Variance inflation factor for SD_FederalWay: 1.47
Variance inflation factor for SD_Issaquah: 1.48
Variance inflation factor for SD_Kent: 1.54
Variance inflation factor for SD_LakeWashington: 1.72
Variance inflation factor for SD_Northshore: 1.3
Variance inflation factor for SD_Renton: 1.31
Variance inflation factor for SD_Seattle: 2.75
Variance inflation factor for SD_Shoreline: 1.51


### With the square foor centric model approach you either end up with high p-values of school districts and high vif of housing features or the inverse, while retaining a relatively low r-squared